In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
import warnings
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

# 데이터 불러오기
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

#모델 함수
def modeling(model,X_train,X_test,y_train,y_test):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    metrics(y_test,pred)

#평가 지표
def metrics(y_test,pred):
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    roc_score = roc_auc_score(y_test,pred,average='macro')
    print('Accuracy : {0:.2f}, Precision : {1:.2f}, Recall : {2:.2f}'.format(accuracy,precision,recall))
    print('F1-score : {0:.2f}, auc : {1:.2f}'.format(f1,roc_score,recall))
    
# Data 특성
df.shape # data n: 284807 , p: 31 (Time, Amount, Class외 28개-)

# Data bar plot 
df.Class.value_counts(normalize=True).plot(kind='bar')
print(df.Class.value_counts(normalize=True)*100) # 매우 불균형한 데이터임을 확인 가능

# 어느정도 balanced한 데이터에 대해서 실험해보고자 데이터 불균형 해소 방법 강구
# 정상건을 Undersampling하는 경우와 사기건을 Oversampling하는 두가지 경우가 있는데, 데이터수가 많을때는 undersampling하는 것이 좋다는 연구 결과에 따라 undersampling 진행


#############################
##### Data Set Generate #####
#############################

# Raw Data
x_data = df.loc[:, 'Time' : 'Amount']
y_data = df.loc[:, 'Class']
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, random_state = 42)

# DF1: 사기율 15%, DF2: 사기율 10%, DF3: 사기율 5%, DF4: 사기율 2%
# Undersampling 을 통해 Data Set 생성
from imblearn.under_sampling import RandomUnderSampler
fraud_class_len=len(df[df['Class']==1])
nonfraud_class_index=df[df['Class']==0].index
random_nonfraud_index=np.random.choice(nonfraud_class_index,24108, replace=False)
fraud_class_index = df[df['Class']==1].index

# DF1 Data Set 생성 (사기율 15%)
DF1_random_nonfraud_index=np.random.choice(nonfraud_class_index,2788, replace=False)
DF1_index = np.concatenate([fraud_class_index, DF1_random_nonfraud_index])
DF1_under_sample=df.loc[DF1_index]

sns.countplot(x='Class', data=DF1_under_sample)

# DF2 sample 만드는 과정 (사기율 10%)
DF2_random_nonfraud_index=np.random.choice(nonfraud_class_index,4428, replace=False)
DF2_index = np.concatenate([fraud_class_index, DF2_random_nonfraud_index])
DF2_under_sample=df.loc[DF2_index]

sns.countplot(x='Class', data=DF1_under_sample)

# DF3 sample 만드는 과정 (사기율 5%)
DF3_random_nonfraud_index=np.random.choice(nonfraud_class_index,9348, replace=False)
DF3_index = np.concatenate([fraud_class_index, DF3_random_nonfraud_index])
DF3_under_sample=df.loc[DF2_index]

sns.countplot(x='Class', data=DF3_under_sample)


# DF4 sample 만드는 과정 (사기율 2%)
DF4_random_nonfraud_index=np.random.choice(nonfraud_class_index,24108, replace=False)
DF4_index = np.concatenate([fraud_class_index, DF4_random_nonfraud_index])
DF4_under_sample=df.loc[DF4_index]

sns.countplot(x='Class', data=DF4_under_sample)

#########################
##### Model Fitting #####
#########################


#################
## 0. Raw data ##
#################
## 0-1. Logistic Regression
LR = LogisticRegression()
print('Logistic Regression for Raw data')
modeling(LR,X_train,X_test,y_train,y_test) 

## 0-2. SVM
print('SVM for Raw data')
svm_clf =SVC(kernel = 'linear')
modeling(svm_clf,X_train,X_test,y_train,y_test)

## 0-3. Random Forest
print('Random Forest for Raw data')
model_rf = RandomForestClassifier(n_estimators = 15)
modeling(model_rf,X_train,X_test,y_train,y_test)

#################
## 4. DF1 data ##
#################
## 0-1. Logistic Regression
DF1_x_data = DF1_under_sample.loc[:, 'Time' : 'Amount']
DF1_y_data = DF1_under_sample.loc[:, 'Class']
DF1_X_train, DF1_X_test, DF1_y_train, DF1_y_test = train_test_split(DF1_x_data, DF1_y_data, test_size = 0.3, random_state = 42)
print('DF1 Logistic Regression') 
modeling(LR,DF1_X_train, DF1_X_test, DF1_y_train, DF1_y_test)

## 0-2. SVM
# SVM, kernel = 'linear'로 선형분리 진행
print('DF1 SVM') 
svm_clf =SVC(kernel = 'linear')
modeling(svm_clf,DF1_X_train, DF1_X_test, DF1_y_train, DF1_y_test )

## 0-3. Random Forest
model_rf = RandomForestClassifier(n_estimators = 15)
print('DF1 Random Forest') 
modeling(model_rf,DF1_X_train, DF1_X_test, DF1_y_train, DF1_y_test )



#################
## 2. DF2 data ##
#################
## 0-1. Logistic Regression
DF2_x_data = DF2_under_sample.loc[:, 'Time' : 'Amount']
DF2_y_data = DF2_under_sample.loc[:, 'Class']
DF2_X_train, DF2_X_test, DF2_y_train, DF2_y_test = train_test_split(DF2_x_data, DF2_y_data, test_size = 0.3, random_state = 42)
print('DF2 Logistic Regression') 
modeling(LR,DF2_X_train, DF2_X_test, DF2_y_train, DF2_y_test)

## 0-2. SVM
# SVM, kernel = 'linear'로 선형분리 진행
print('DF2 SVM') 
svm_clf =SVC(kernel = 'linear')
modeling(svm_clf,DF2_X_train, DF2_X_test, DF2_y_train, DF2_y_test)

## 0-3. Random Forest
model_rf = RandomForestClassifier(n_estimators = 15)
print('DF2 Random Forest') 
modeling(model_rf,DF2_X_train, DF2_X_test, DF2_y_train, DF2_y_test)

#################
## 3. DF3 data ##
#################
## 0-1. Logistic Regression
DF3_x_data = DF3_under_sample.loc[:, 'Time' : 'Amount']
DF3_y_data = DF3_under_sample.loc[:, 'Class']
DF3_X_train, DF3_X_test, DF3_y_train, DF3_y_test = train_test_split(DF3_x_data, DF3_y_data, test_size = 0.3, random_state = 42)
print('DF3 Logistic Regression') 
modeling(LR,DF3_X_train, DF3_X_test, DF3_y_train, DF3_y_test)

## 0-2. SVM
# SVM, kernel = 'linear'로 선형분리 진행
print('DF3 SVM') 
svm_clf =SVC(kernel = 'linear')
modeling(svm_clf,DF3_X_train, DF3_X_test, DF3_y_train, DF3_y_test )

## 0-3. Random Forest
model_rf = RandomForestClassifier(n_estimators = 15)
print('DF3 Random Forest') 
modeling(model_rf,DF3_X_train, DF3_X_test, DF3_y_train, DF3_y_test)


#################
## 4. DF4 data ##
#################
## 0-1. Logistic Regression
DF4_x_data = DF4_under_sample.loc[:, 'Time' : 'Amount']
DF4_y_data = DF4_under_sample.loc[:, 'Class']
DF4_X_train, DF4_X_test, DF4_y_train, DF4_y_test = train_test_split(DF4_x_data, DF4_y_data, test_size = 0.3, random_state = 42)
print('DF4 Logistic Regression') 
modeling(LR,DF4_X_train, DF4_X_test, DF4_y_train, DF4_y_test)

## 0-2. SVM
# SVM, kernel = 'linear'로 선형분리 진행
print('DF4 SVM') 
svm_clf =SVC(kernel = 'linear')
modeling(svm_clf,DF4_X_train, DF4_X_test, DF4_y_train, DF4_y_test)

## 0-3. Random Forest
model_rf = RandomForestClassifier(n_estimators = 15)
print('DF4 Random Forest') 
modeling(model_rf,DF4_X_train, DF4_X_test, DF4_y_train, DF4_y_test)
